In [2]:
import pandas as pd
import numpy as np
#from scipy.linalg import svd
from sklearn.decomposition import randomized_svd

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim.corpora as corpora


Initializing our dataframe and adding the serious column


In [3]:
df = pd.read_csv('2022VAERSData.csv', encoding='cp1252', low_memory=False)
df.dropna(subset=["SYMPTOM_TEXT"], inplace=True, axis=0)
df = df.reset_index()


def is_serious(row):
    columns = ["DIED", "ER_VISIT", "HOSPITAL", "DISABLE"]
    for val in row[columns]:
        if isinstance(val, str) and val.upper() == "Y":
            return True
    return False


df["SERIOUS"] = df.apply(is_serious, axis=1)
documents = df["SYMPTOM_TEXT"]

In [4]:
stop_words = set(
    stopwords.words('english') + [
        ".", ":", ";", "(", ")", ",", "#", "'", "\"", '!', '?', '$', '%', '&',
        "''", "``"
    ])

porter = PorterStemmer()


def stopword_remover(lst):
    return [word for word in lst if word not in stop_words]


def stemmer(lst):
    return [porter.stem(word) for word in lst if word]


def text_preprocess(d):
    tokens = np.array([word_tokenize(i) for i in d], dtype=object)
    tok_fil = [stopword_remover(doc) for doc in tokens]
    tok_stem = [stemmer(doc) for doc in tok_fil]
    return tok_stem


documents = text_preprocess(
    documents)  # remove stopwords and punctuation, convert words to stems


In [5]:
vectorizer = TfidfVectorizer(tokenizer=lambda i: i, lowercase=False)
doc_term_matrix = vectorizer.fit_transform(documents).todense()
vocab = np.array(vectorizer.get_feature_names_out())
vocab[20000:20010]

array(['hug', 'huge', 'hugh', 'hum', 'humalog', 'human', 'humera',
       'humeru', 'humidifi', 'humili'], dtype=object)

In [6]:
u, s, v = randomized_svd(doc_term_matrix, 10, random_state=None)

In [7]:
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-15:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [8]:
show_topics(v)  # showing top 15 words from all(10) topics


['report patient vaccin unknown the covid-19 dose receiv medic i inform no number pain',
 'vendor mobil might dure record excurs review intern store temperatur found depart possibl health',
 'i day pain bodi arm start fever headach chest sever hour ach left symptom',
 'covid breakthrough posit case hospit test vaccin + admit pt fulli cough infect contact',
 'expir vaccin moderna administ given use mrna-1273 i dose on pfizer beyond vial product',
 'pain fever arm site headach ach chill inject bodi fatigu sore chest swell left',
 'pfizer fj6369 given system staff ok instruct use temp excurs lot compani hospit pt',
 'breakthrough case report arm pain fj6369 pfizer site inject system ok staff instruct temp',
 'hospit patient breakthrough 12/22/21 beyond the date vaccin admit receiv bnt162b2 number left she',
 'expir date beyond administ use 12/22/21 posit limit test vaccin covid bnt162b2 vial onset']

s is the matrix topics x topics, size 10 since we have 10 topics and full_matrics=False, so 1x10 matrix instead of 10x10


In [9]:
s

array([36.35022468, 28.42038069, 22.66211709, 19.0316914 , 17.31889475,
       15.66227467, 15.57815637, 15.13166503, 14.01848067, 13.55911358])

u is documents(rows) x topics(columns)


In [10]:
u

array([[ 8.96332425e-03,  1.11123505e-03, -5.50606415e-03, ...,
         8.44128003e-04,  2.24248103e-03,  9.51644968e-03],
       [ 1.25807046e-02, -1.61937558e-03, -6.33539313e-03, ...,
        -2.52133029e-04,  1.75638406e-03, -4.14285670e-03],
       [ 1.29308344e-02, -1.55531618e-03, -5.93376475e-03, ...,
        -2.06188777e-03, -3.55814360e-03, -6.59699722e-03],
       ...,
       [ 2.34207995e-03,  6.66212631e-05,  5.53270165e-04, ...,
        -5.44641484e-04,  1.30629993e-03,  9.94180464e-04],
       [ 2.31824661e-03,  6.93989649e-05,  5.75267769e-04, ...,
        -6.81685322e-04,  1.39830369e-03,  9.85663875e-04],
       [ 2.13217731e-03,  6.06385657e-05,  5.04435266e-04, ...,
        -4.97249142e-04,  1.19423189e-03,  9.08330606e-04]])

v is topic(rows) x terms(columns) matrix


In [11]:
v

array([[ 2.05755129e-05,  6.02442548e-05,  4.96746512e-05, ...,
         4.44020728e-04,  1.04282458e-05,  2.42914351e-04],
       [-3.94945290e-06,  5.76653262e-06, -8.31564742e-06, ...,
        -7.13624509e-05, -1.26396644e-06, -4.57448257e-05],
       [ 2.03065821e-05,  1.20711920e-04, -1.98983869e-05, ...,
        -2.63776724e-04, -2.03461802e-06,  1.45437866e-04],
       ...,
       [ 2.09492474e-05,  3.41659774e-05,  1.40016716e-05, ...,
        -5.69227744e-05, -1.50920155e-06,  1.25998478e-04],
       [ 1.70364717e-05,  6.79360356e-05, -1.90570818e-06, ...,
        -4.21993757e-04, -5.52168950e-06,  1.79706161e-04],
       [ 1.20846410e-05,  8.59092511e-05,  4.12041721e-05, ...,
        -7.49818139e-04, -8.98669533e-06,  7.42280306e-05]])

## Latent Dirichlet Allocation


In [12]:
from gensim.utils import simple_preprocess


def sent_to_words(sentences):
    for sentence in sentences:
        yield (simple_preprocess(str(sentence), deacc=True))


def remove_stopwords(texts):
    return [[
        word for word in simple_preprocess(str(doc)) if word not in stop_words
    ] for doc in texts]


data = documents
data_words = list(sent_to_words(data))
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['thi', 'spontan', 'report', 'receiv', 'pharmacist', 'refer', 'patient', 'unknown', 'age', 'gender', 'inform', 'regard', 'patient', 'medic', 'histori', 'concurr', 'condit', 'concomit', 'medic', 'provid', 'dec', 'patient', 'vaccin', 'expir', 'hpv', 'rl', 'vlp', 'vaccin', 'yeast', 'gardasil']


In [13]:
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]
corpus

[[(0, 1),
  (1, 2),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 3),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 3),
  (29, 1),
  (30, 1),
  (31, 2),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 3),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 5),
  (45, 1),
  (46, 1),
  (47, 1)],
 [(0, 1),
  (1, 1),
  (4, 1),
  (6, 1),
  (7, 1),
  (8, 2),
  (9, 1),
  (10, 2),
  (11, 1),
  (12, 3),
  (19, 1),
  (21, 2),
  (22, 1),
  (24, 1),
  (26, 3),
  (28, 5),
  (32, 3),
  (35, 13),
  (37, 1),
  (39, 1),
  (40, 1),
  (42, 4),
  (43, 3),
  (44, 8),
  (48, 2),
  (49, 3),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 3),
  (59, 1),
  (60, 1),
  (61, 4),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 3),
  (

In [14]:
from gensim.models import LdaMulticore

lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=10)

doc_lda = lda_model[corpus]
lda_model.print_topics()

[(0,
  '0.023*"vaccin" + 0.016*"patient" + 0.014*"report" + 0.012*"day" + 0.011*"covid" + 0.009*"dose" + 0.008*"start" + 0.008*"unknown" + 0.008*"medic" + 0.008*"pain"'),
 (1,
  '0.030*"patient" + 0.028*"vaccin" + 0.026*"report" + 0.025*"covid" + 0.016*"day" + 0.013*"dose" + 0.012*"receiv" + 0.011*"unknown" + 0.010*"pain" + 0.010*"medic"'),
 (2,
  '0.036*"patient" + 0.025*"report" + 0.020*"vaccin" + 0.017*"dose" + 0.015*"pain" + 0.014*"covid" + 0.012*"non" + 0.011*"receiv" + 0.009*"inform" + 0.008*"medic"'),
 (3,
  '0.025*"vaccin" + 0.025*"patient" + 0.024*"report" + 0.020*"unknown" + 0.018*"pain" + 0.017*"arm" + 0.015*"covid" + 0.014*"receiv" + 0.014*"dose" + 0.012*"medic"'),
 (4,
  '0.039*"patient" + 0.027*"covid" + 0.026*"vaccin" + 0.024*"dose" + 0.023*"report" + 0.018*"unknown" + 0.017*"receiv" + 0.014*"medic" + 0.010*"thi" + 0.009*"year"'),
 (5,
  '0.024*"covid" + 0.019*"vaccin" + 0.014*"patient" + 0.013*"pain" + 0.012*"day" + 0.011*"report" + 0.008*"medic" + 0.007*"dose" + 0.007*

In [15]:
for doc in range(5):
    print(f"Document {doc}: ", end="")
    for topic, prob in doc_lda[doc]:
        print(f"Topic: {topic} Probability: {prob*100:.2f}% ", end="")
    print()


Document 0: Topic: 7 Probability: 98.55% 
Document 1: Topic: 2 Probability: 29.11% Topic: 7 Probability: 55.13% Topic: 8 Probability: 15.40% 
Document 2: Topic: 2 Probability: 10.11% Topic: 7 Probability: 41.09% Topic: 8 Probability: 48.33% 
Document 3: Topic: 2 Probability: 44.53% Topic: 4 Probability: 54.39% 
Document 4: Topic: 3 Probability: 36.90% Topic: 4 Probability: 62.45% 


In [39]:
import pyLDAvis.gensim_models
import pyLDAvis
import warnings

warnings.filterwarnings('ignore')

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.012355  0.013316       1        1  20.633153
7      0.039195 -0.001004       2        1  16.833270
5     -0.045592 -0.023641       3        1  15.894960
0     -0.031833  0.033853       4        1  11.446143
4      0.028728 -0.014027       5        1   9.006749
2      0.006112  0.003150       6        1   8.633003
8      0.014712  0.011497       7        1   4.824458
6     -0.001197 -0.016378       8        1   4.293576
9     -0.009505 -0.005237       9        1   4.243890
1     -0.012975 -0.001529      10        1   4.190798, topic_info=        Term          Freq         Total Category  logprob  loglift
28   patient  42135.000000  42135.000000  Default  30.0000  30.0000
44    vaccin  41804.000000  41804.000000  Default  29.0000  29.0000
35    report  36419.000000  36419.000000  Default  28.0000  28.0000
71     covid  31125.000000  31125.000000  Default  27.0000  27.0000
73      dose  24072.000000  24072.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
170  describ    289.919683  10555.344347  Topic10  -5.4589  -0.4225
194      non    290.423991  11070.422632  Topic10  -5.4572  -0.4684
134      arm    294.254521  12475.469034  Topic10  -5.4441  -0.5748
79    follow    280.485144   9752.031865  Topic10  -5.4920  -0.3764
216     year    272.122695   9204.982622  Topic10  -5.5223  -0.3490

[891 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
1853       1  0.134918  accident
1853       2  0.240665  accident
1853       3  0.021879  accident
1853       4  0.040111  accident
1853       5  0.393815  accident
...      ...       ...       ...
970        7  0.064164    zyrtec
970        8  0.040102    zyrtec
970        9  0.157735    zyrtec
970       10  0.080204    zyrtec
15404      5  0.440457        ºf

[3023 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 6, 1, 5, 3, 9, 7, 10, 2])

In [43]:
def get_topic(row):
    topics_for_row = doc_lda[row.name]
    topics_for_row.sort(key=lambda a: -a[1])
    topic, highest_prob = topics_for_row[0]
    return topic

df["TOPIC"] = df.apply(get_topic, axis=1)

In [41]:
df[["SYMPTOM_TEXT", "TOPIC"]][0:10]

,SYMPTOM_TEXT,TOPIC
0,This spontaneous report was received from a ph...,7
1,SUSPECTED CLINICAL VACCINATION FAILURE; SUSPEC...,7
2,SUSPECTED CLINICAL VACCINATION FAILURE; SUSPEC...,8
3,Irregular menstrual cycle.; period extremely h...,4
4,Breakthrough and heavy periods; longer period ...,4
5,tired; Patient reports feeling achy; chills; t...,3
6,diarrheal discharge of both goop and intestina...,0
7,Now Lymph node swelling in armpit closest to j...,3
8,my torso was covered with a rash/rash spreadof...,3
9,pyrexia; This is a spontaneous report received...,2


In [63]:
from collections import Counter, defaultdict
topics = Counter(df["TOPIC"])
topics_serious = defaultdict(int)

for index, row in df.iterrows():
  if row["SERIOUS"]:
    topics_serious[row["TOPIC"]] += 1

topics = sorted(([(topic, count) for topic, count in topics.items()]))
topics_serious = sorted(([(topic, count) for topic, count in topics_serious]))

print(topics)
print(topics_serious)


Counter({5: 5490, 0: 4744, 3: 3184, 7: 3106, 4: 2182, 1: 1597, 9: 1380, 2: 1217, 6: 930, 8: 881})
defaultdict(<class 'int'>, {3: 113, 5: 2643, 9: 50, 0: 363, 6: 157, 1: 293, 8: 189, 2: 107, 4: 252, 7: 165})
